In [1]:
import os
import folium
import numpy as np
import pandas as pd
import json
import time
from decimal import Decimal
from folium import plugins
import geopy
from geopy.geocoders import Nominatim

In [2]:
import geopandas as gpd


assert 'naturalearth_lowres' in gpd.datasets.available
datapath = gpd.datasets.get_path('naturalearth_lowres')
gdf = gpd.read_file(datapath)

In [3]:
new_data = pd.read_csv("Inch.csv")
new_data = new_data.drop('Unnamed: 0',axis=1)

In [4]:
d = {}
for i in range(8):
    d[i] = new_data[new_data["year"]==2010+i]
    d[i]=d[i].reset_index().drop("index",axis=1)
    

In [5]:
# load GeoJSON
with open('Boundaries - ZIP Codes.geojson', 'r') as jsonFile:
    data_ = json.load(jsonFile)
tmp = data_

# remove ZIP codes not in our dataset
geozips = []
A=[]
for i in range(len(new_data["Zip"].unique())):
    A.append(int((tmp['features'][i]['properties']['zip'])))
    if A[i] in (new_data["Zip"].unique()):
        tmp['features'][i]['properties']['zip'] = int((tmp['features'][i]['properties']['zip']))
        geozips.append(tmp['features'][i])
        
# creating new JSON object
new_json = dict.fromkeys(['type','features'])
new_json['type'] = 'FeatureCollection'
new_json['features'] = geozips

# save JSON object as updated-file
open("time_updated-file.json", "w").write(
    json.dumps(new_json, sort_keys=True, indent=4, separators=(',', ': '))
)

5709037

In [6]:
n_periods, n_sample = 8, 5
datetime_index = pd.date_range('2010-01-01',"2018-01-01", freq='A-JAN')
dt_index_epochs = datetime_index.astype(int) // 10**9
dt_index = dt_index_epochs.astype('U10')

dt_index

Index(['1264896000', '1296432000', '1327968000', '1359590400', '1391126400',
       '1422662400', '1454198400', '1485820800'],
      dtype='object')

In [14]:
new_data

,Zip,year,Violation risk,Count
0,60601,2010,0.985348,273
1,60601,2011,1.061093,311
2,60601,2012,1.087649,251
3,60601,2013,1.291536,319
4,60601,2014,1.044910,334
...,...,...,...,...
468,60827,2014,1.588235,17
469,60827,2015,0.800000,10
470,60827,2016,1.266667,15
471,60827,2017,1.111111,9


In [15]:
styledata = {}

for ZIP in new_data.Zip:
    df = pd.DataFrame(
        {'color': np.random.normal(size=n_periods),
         'opacity': np.random.normal(size=n_periods)},
        index=dt_index
    )
    df = df.cumsum()
    df.sample(n_sample, replace=False).sort_index()
    styledata[ZIP] = df

In [16]:
max_color, min_color, max_opacity, min_opacity = 0, 0, 0, 0

for country, data in styledata.items():
    max_color = max(max_color, data['color'].max())
    min_color = min(max_color, data['color'].min())
    max_opacity = max(max_color, data['opacity'].max())
    max_opacity = min(max_color, data['opacity'].max())

In [17]:
from branca.colormap import linear

cmap = linear.PuRd_09.scale(min_color, max_color)


def norm(x):
    return (x - x.min()) / (x.max() - x.min())


for country, data in styledata.items():
    data['color'] = data['color'].apply(cmap)
    data['opacity'] = norm(data['opacity'])

In [18]:
styledict = {
    str(country): data.to_dict(orient='index') for
    country, data in styledata.items()
}

In [19]:
styledict = {int(k):v for k,v in styledict.items()}

In [20]:
styledict[60601]

{'1264896000': {'color': '#cc8dc4', 'opacity': 0.8426691980302782},
 '1296432000': {'color': '#d47cbc', 'opacity': 0.8460111186166387},
 '1327968000': {'color': '#da70b6', 'opacity': 0.5508307000954493},
 '1359590400': {'color': '#e72f8e', 'opacity': 0.8330841986008217},
 '1391126400': {'color': '#e24fa2', 'opacity': 1.0},
 '1422662400': {'color': '#e6308f', 'opacity': 0.7166278590464846},
 '1454198400': {'color': '#a70548', 'opacity': 0.33608645452159125},
 '1485820800': {'color': '#d61865', 'opacity': 0.0}}

In [21]:
def create_map(table, zips, mapped_feature, add_text = ''):
    # reading of the updated GeoJSON file
    geo = r'time_updated-file.json'
    # initiating a Folium map with Chicago's longitude and latitude
    coord = [41.88270035458986, -87.63663755997726]
    m = folium.Map(location = coord, tiles='cartodbpositron',zoom_start = 10)
    # creating a choropleth map
    g = TimeSliderChoropleth(geo,styledict=styledict,).add_to(m)
    m.choropleth(
        geo_data = geo,
        fill_opacity = 0.7,
        line_opacity = 0.2,
        data = table,
        # refers to which key within the GeoJSON to map the ZIP code to
        key_on = 'feature.properties.zip',
        # first element contains location information, second element contains feature of interest
        columns = [zips, mapped_feature],
        fill_color = 'BuPu',
        legend_name = (' ').join(mapped_feature.split('_')).title() + ' Number of facilites ' + add_text + ' Across Chigaco'
    )
    folium.LayerControl().add_to(m)
    # save map with filename based on the feature of interest
    m.save(outfile = mapped_feature + '_map.html')
    return(m)

In [22]:
create_map(new_data,"Zip","Violation risk")

NameError: name 'TimeSliderChoropleth' is not defined

In [35]:

# reading of the updated GeoJSON file
geo = r'time_updated-file.json'
# initiating a Folium map with Chicago's longitude and latitude
coord = [41.88270035458986, -87.63663755997726]
m = folium.Map(location = coord, tiles='stamentoner',zoom_start = 10)
# creating a choropleth map
m.choropleth(
    geo_data = geo,
    fill_opacity = 0.7,
    line_opacity = 0.2,  
    data = d[0],
    # refers to which key within the GeoJSON to map the ZIP code to
    key_on = 'feature.properties.zip',
    # first element contains location information, second element contains feature of interest
    columns = ["Zip","Violation risk"],
    fill_color = 'BuPu',
    legend_name = (' ').join("Violation risk".split('_')).title() + ' Number of facilites Across Chigaco'
)
m.choropleth(
    geo_data = geo,
    fill_opacity = 0.7,
    line_opacity = 0.2,  
    data = d[1],
    # refers to which key within the GeoJSON to map the ZIP code to
    key_on = 'feature.properties.zip',
    # first element contains location information, second element contains feature of interest
    columns = ["Zip","Violation risk"],
    fill_color = 'BuPu',
    legend_name = (' ').join("Violation risk".split('_')).title() + ' Number of facilites Across Chigaco'
)
m.choropleth(
    geo_data = geo,
    fill_opacity = 0.7,
    line_opacity = 0.2,  
    data = d[2],
    # refers to which key within the GeoJSON to map the ZIP code to
    key_on = 'feature.properties.zip',
    # first element contains location information, second element contains feature of interest
    columns = ["Zip","Violation risk"],
    fill_color = 'BuPu',
    overlay = True,
    legend_name = (' ').join("Violation risk".split('_')).title() + ' Number of facilites Across Chigaco'
)
m.choropleth(
    geo_data = geo,
    fill_opacity = 0.7,
    line_opacity = 0.2,  
    data = d[3],
    # refers to which key within the GeoJSON to map the ZIP code to
    key_on = 'feature.properties.zip',
    # first element contains location information, second element contains feature of interest
    columns = ["Zip","Violation risk"],
    fill_color = 'BuPu',
    legend_name = (' ').join("Violation risk".split('_')).title() + ' Number of facilites Across Chigaco'
)
m.choropleth(
    geo_data = geo,
    fill_opacity = 0.7,
    line_opacity = 0.2,  
    data = d[4],
    # refers to which key within the GeoJSON to map the ZIP code to
    key_on = 'feature.properties.zip',
    # first element contains location information, second element contains feature of interest
    columns = ["Zip","Violation risk"],
    fill_color = 'BuPu',
    legend_name = (' ').join("Violation risk".split('_')).title() + ' Number of facilites Across Chigaco'
)
m.choropleth(
    geo_data = geo,
    fill_opacity = 0.7,
    line_opacity = 0.2,  
    data = d[5],
    # refers to which key within the GeoJSON to map the ZIP code to
    key_on = 'feature.properties.zip',
    # first element contains location information, second element contains feature of interest
    columns = ["Zip","Violation risk"],
    fill_color = 'BuPu',
    legend_name = (' ').join("Violation risk".split('_')).title() + ' Number of facilites Across Chigaco'
)
m.choropleth(
    geo_data = geo,
    fill_opacity = 0.7,
    line_opacity = 0.2,  
    data = d[6],
    # refers to which key within the GeoJSON to map the ZIP code to
    key_on = 'feature.properties.zip',
    # first element contains location information, second element contains feature of interest
    columns = ["Zip","Violation risk"],
    fill_color = 'BuPu',
    legend_name = (' ').join("Violation risk".split('_')).title() + ' Number of facilites Across Chigaco'
)
m.choropleth(
    geo_data = geo,
    fill_opacity = 0.7,
    line_opacity = 0.2,  
    data = d[7],
    # refers to which key within the GeoJSON to map the ZIP code to
    key_on = 'feature.properties.zip',
    # first element contains location information, second element contains feature of interest
    columns = ["Zip","Violation risk"],
    fill_color = 'BuPu',
    legend_name = (' ').join("Violation risk".split('_')).title() + ' Number of facilites Across Chigaco'
)

folium.LayerControl().add_to(m)
m.save("Bite.html")